# Evo-Bayes-Agent

In [1]:
from Ionomy import IonPanda, BitTA
from decouple import config
import pandas as pd
import pandas_ta as ta

In [2]:
# MARKET = 'usd-btc'
# CURRENCY = 'btc'
# BASE = 'usd'
# TIME = 'hour'
# LIMIT = 10_000

In [3]:
# bta = BitTA(config('TREX_KEY'), config('TREX_SECRET'))
# ionpd = IonPanda(config('IONOMY_KEY'), config('IONOMY_SECRET'))
# bta.update(CURRENCY, BASE, TIME, LIMIT)
# df = bta.df
# df.to_csv('./btc-usd', index=False)

In [4]:
df = pd.read_csv('./btc-usd')

In [5]:
df

,time,close,high,low,open,volumebtc,volume,date
0,1586433600,7313.89,7313.89,7110.00,7195.43,69.920,505017.77,2020-04-09
1,1586437200,7349.90,7362.39,7290.07,7313.89,47.340,346817.26,2020-04-09
2,1586440800,7327.00,7363.00,7301.92,7349.90,51.100,374672.75,2020-04-09
3,1586444400,7290.00,7331.07,7259.15,7327.00,44.690,325767.63,2020-04-09
4,1586448000,7290.00,7304.50,7275.31,7290.00,19.200,139998.07,2020-04-09
...,...,...,...,...,...,...,...,...
9996,1564819200,10790.00,10837.41,10763.88,10828.03,11.630,125769.40,2019-08-03
9997,1564822800,10764.66,10886.99,10740.00,10790.00,24.250,262521.18,2019-08-03
9998,1564826400,10793.64,10800.00,10754.23,10764.66,1.255,13526.50,2019-08-03
9999,1564830000,10758.49,10799.74,10753.05,10793.64,4.668,50290.94,2019-08-03


In [6]:
from neuroevolution import DemiChad

Using TensorFlow backend.


In [7]:
demichad = DemiChad(df)

In [8]:
# demichad.meta_evolve()

In [ ]:
demichad.evolve(100)

32


In [ ]:
demichad.omega()

In [ ]:
demichad.omega_chad.sell_history

In [ ]:
demichad.omega_chad.sell_history.profit.sum()/len(demichad.omega_chad.sell_history)

In [ ]:
demichad.omega_chad.sell_history.profit.sum()

In [ ]:
demichad.plot()